<a href="https://colab.research.google.com/github/adrianacupp/NLP_with_Transformers/blob/main/BART_Relation_Classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adracupp","key":"e74d1222705292f7c525d7e2a435b34f"}'}

In [2]:
#2. Series of commands to set-up for download

!ls -lha kaggle.json
!pip install -q kaggle # installing the kaggle package
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder
!pwd # checking the present working directory

-rw-r--r-- 1 root root 64 Aug  8 09:59 kaggle.json
/content


In [3]:
#3. giving rw access (if 401-nathorized)

!chmod 600 ~/.kaggle/kaggle.json

In [4]:
#4. Sanity check if able to access kaggle
!kaggle datasets list

ref                                                        title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
myrios/cost-of-living-index-by-country-by-number-2024      Cost of Living Index by Country                      3KB  2024-07-19 06:25:42           2244         29  1.0              
waqi786/cats-dataset                                       🐾 Cats Dataset                                       6KB  2024-07-31 06:12:49           2108         43  1.0              
muhammadehsan000/olympic-historical-dataset-1896-2020      Olympic Historical Dataset (1896 - 2022)            27MB  2024-08-06 16:46:08           1141         26  1.0              
abhayayare/employee-data-simulation-it-industry            Employee Data Simulation: IT In

In [5]:
!kaggle datasets download -d daishinkan002/new-york-times-relation-extraction-dataset

Dataset URL: https://www.kaggle.com/datasets/daishinkan002/new-york-times-relation-extraction-dataset
License(s): unknown
 55% 5.00M/9.02M [00:00<00:00, 30.7MB/s]
100% 9.02M/9.02M [00:00<00:00, 47.3MB/s]


In [6]:
!pwd

/content


In [7]:
!ls -lha

total 9.1M
drwxr-xr-x 1 root root 4.0K Aug  8 10:00 .
drwxr-xr-x 1 root root 4.0K Aug  8 08:41 ..
drwxr-xr-x 4 root root 4.0K Aug  6 13:32 .config
drwxr-xr-x 2 root root 4.0K Aug  8 08:44 dataset
drwx------ 6 root root 4.0K Aug  8 08:46 drive
drwxr-xr-x 2 root root 4.0K Aug  8 09:51 .ipynb_checkpoints
-rw-r--r-- 1 root root   64 Aug  8 09:59 kaggle.json
-rw-r--r-- 1 root root 9.1M Jul 31  2021 new-york-times-relation-extraction-dataset.zip
drwxr-xr-x 1 root root 4.0K Aug  6 13:32 sample_data


In [9]:
!unzip -f new-york-times-relation-extraction-dataset.zip

Archive:  new-york-times-relation-extraction-dataset.zip


In [10]:
# Function to extract relations from a JSON object
def extract_relations(json_line):
    relations = []
    sent_text = json_line['sentText']
    for relation in json_line['relationMentions']:
        em1_text = relation['em1Text']
        em2_text = relation['em2Text']
        label = relation['label']
        relations.append((sent_text, em1_text, em2_text, label))
    return relations

In [11]:
import json

# Define the file path
input_file = 'dataset/train.json'

# Initialize a list to hold the data
data = []

# Initialize a counter for the number of lines processed
count = 0

# Read the JSON file
with open(input_file, 'r') as file:
    for line in file:
        json_line = json.loads(line)  # Load JSON object from the line
        relations = extract_relations(json_line)  # Extract relations
        data.extend(relations)  # Add extracted relations to the data list
        count += 1  # Increment the counter

In [12]:
# Print the number of lines processed and the first few extracted relations
print(f"Processed {count} lines.")
print(f"First few extracted relations: {data[:5]}")

Processed 56196 lines.
First few extracted relations: [('Massachusetts ASTON MAGNA Great Barrington ; also at Bard College , Annandale-on-Hudson , N.Y. , July 1-Aug .', 'Annandale-on-Hudson', 'Bard College', '/location/location/contains'), ('North Carolina EASTERN MUSIC FESTIVAL Greensboro , June 25-July 30 .', 'North Carolina', 'Greensboro', '/location/location/contains'), ("It will be the final movie credited to Debra Hill , a film producer and native of Haddonfield , who produced '' Halloween '' and was considered a pioneering woman in film .", 'Debra Hill', 'Haddonfield', '/people/person/place_of_birth'), ("In a 3-0 victory over the Crew on Saturday in Columbus , Ohio , goalkeeper Zach Wells stopped Kyle Martino 's penalty kick , only the third unsuccessful penalty in 20 attempts in M.L.S. this season .", 'Ohio', 'Columbus', '/location/location/contains'), ("The United States ambassador to Mexico , Tony Garza , said in a statement that he had directed the American Consulate in Nuev

In [13]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

# Convert the extracted data to a pandas DataFrame
df = pd.DataFrame(data, columns=['sentence', 'entity1', 'entity2', 'label'])

output_path = '/content/drive/MyDrive/Colab Notebooks/extracted_relations.csv'

df.to_csv(output_path, index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
print(df.head())
print(df.info())

                                            sentence              entity1  \
0  Massachusetts ASTON MAGNA Great Barrington ; a...  Annandale-on-Hudson   
1  North Carolina EASTERN MUSIC FESTIVAL Greensbo...       North Carolina   
2  It will be the final movie credited to Debra H...           Debra Hill   
3  In a 3-0 victory over the Crew on Saturday in ...                 Ohio   
4  The United States ambassador to Mexico , Tony ...               Mexico   

        entity2                          label  
0  Bard College    /location/location/contains  
1    Greensboro    /location/location/contains  
2   Haddonfield  /people/person/place_of_birth  
3      Columbus    /location/location/contains  
4  Nuevo Laredo    /location/location/contains  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94222 entries, 0 to 94221
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  94222 non-null  object
 1   entity1   94222 

In [15]:
#Change the first letter to an empty string in the label column
df['label'] = df['label'].str[1:]
df['label'] = df['label'].str.replace('/', '_')
df.head()

,sentence,entity1,entity2,label
0,Massachusetts ASTON MAGNA Great Barrington ; a...,Annandale-on-Hudson,Bard College,location_location_contains
1,North Carolina EASTERN MUSIC FESTIVAL Greensbo...,North Carolina,Greensboro,location_location_contains
2,It will be the final movie credited to Debra H...,Debra Hill,Haddonfield,people_person_place_of_birth
3,In a 3-0 victory over the Crew on Saturday in ...,Ohio,Columbus,location_location_contains
4,"The United States ambassador to Mexico , Tony ...",Mexico,Nuevo Laredo,location_location_contains


In [16]:
# Print the unique labels in the label column
unique_labels = df['label'].unique()
sorted(unique_labels)

['business_company_advisors',
 'business_company_founders',
 'business_company_industry',
 'business_company_major_shareholders',
 'business_company_place_founded',
 'business_company_shareholder_major_shareholder_of',
 'business_person_company',
 'location_administrative_division_country',
 'location_country_administrative_divisions',
 'location_country_capital',
 'location_location_contains',
 'location_neighborhood_neighborhood_of',
 'people_deceased_person_place_of_death',
 'people_ethnicity_geographic_distribution',
 'people_ethnicity_people',
 'people_person_children',
 'people_person_ethnicity',
 'people_person_nationality',
 'people_person_place_lived',
 'people_person_place_of_birth',
 'people_person_profession',
 'people_person_religion',
 'sports_sports_team_location',
 'sports_sports_team_location_teams']

In [17]:
df.columns

Index(['sentence', 'entity1', 'entity2', 'label'], dtype='object')

In [18]:
df['sentence_length'] = df['sentence'].apply(lambda x: len(x.split()))
avg_sentence_length = df['sentence_length'].mean()
print(f"Average sentence length: {avg_sentence_length} words")

Average sentence length: 39.01351064507227 words


# Preprocess

In [19]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer, AutoModelForSeq2SeqLM, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split

In [20]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [21]:
# Initialize the tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [23]:
# Create label encoder
label_encoder = {label: i for i, label in enumerate(df['label'].unique())}
label_decoder = {i: label for label, i in label_encoder.items()}

In [24]:
# Check the maximum length of the tokenized labels
label_lengths = [len(tokenizer.encode(label, add_special_tokens=True)) for label in df['label'].unique()]
max_label_length = max(label_lengths)
print(f"Maximum label length: {max_label_length}")

Maximum label length: 15


In [25]:
# in BART model the labels (relationship types) are treated as sequences (strings)
# that need to be tokenized just like the input sentence
def preprocess_data(row):
    sentence = row['sentence']
    label = row['label']

    # Tokenize input sentence
    encoded_sentence = tokenizer(
        sentence,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    ).to(device)

    # Encode the label as the target sequence
    encoded_label = tokenizer(
        label,
        add_special_tokens=True,
        max_length=15,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    ).to(device)

    return {
        'input_ids': encoded_sentence['input_ids'].squeeze(),
        'attention_mask': encoded_sentence['attention_mask'].squeeze(),
        'labels': encoded_label['input_ids'].squeeze()
    }
  #labels: sequence of token IDs representing the target relationship label

In [26]:
#apply preprocess to dataset
train_data = train_df.apply(preprocess_data, axis=1).tolist()
val_data = val_df.apply(preprocess_data, axis=1).tolist()

In [ ]:
# Create custom dataset class
class RelationshipDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
# Create dataset and dataloader
train_dataset = RelationshipDataset(train_data)
val_dataset = RelationshipDataset(val_data)
train_dataloader = DataLoader(train_dataset, batch_size=24, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Define Model

In [ ]:
from transformers import AutoModelForSeq2SeqLM

In [ ]:
class RelationshipExtractionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.bart = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large')

    def forward(self, input_ids, attention_mask, labels):
        outputs = self.bart(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        return outputs.loss, outputs.logits

In [ ]:
model = RelationshipExtractionModel().to(device)

In [ ]:
# Training configuration
num_epochs = 7
learning_rate = 2.5e-5
gradient_accumulation_steps = 1
max_grad_norm = 1.0

In [ ]:
# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.1)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=8000,
    num_training_steps=total_steps
)

# Training Loop

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(model, dataloader, device, tokenizer, label_decoder):
    model.eval()
    total_preds = []
    total_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Generate predictions
            outputs = model.generate(input_ids, attention_mask=attention_mask)
            preds = [tokenizer.decode(g, skip_special_tokens=True) for g in outputs]
            labels = [tokenizer.decode(l, skip_special_tokens=True) for l in labels]

            # Collect predictions and labels
            total_preds.extend(preds)
            total_labels.extend(labels)

    # Convert labels and predictions to a common format
    total_preds = [label_decoder.get(p, p) for p in total_preds]
    total_labels = [label_decoder.get(l, l) for l in total_labels]

    # Calculate metrics
    precision = precision_score(total_labels, total_preds, average='weighted')
    recall = recall_score(total_labels, total_preds, average='weighted')
    f1 = f1_score(total_labels, total_preds, average='weighted')

    return precision, recall, f1


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AdamW
from torch.utils.data import DataLoader
from tqdm.auto import tqdm


for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        # Access loss from the tuple
        if isinstance(outputs, tuple):
            loss = outputs[0]  # The first element of the tuple is the loss
        else:
            loss = outputs.loss  # For a ModelOutput object

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': total_loss / (progress_bar.n + 1)})

    # Calculate validation metrics
    val_precision, val_recall, val_f1 = evaluate(model, val_dataloader, device, tokenizer, label_decoder)
    print(f"Epoch {epoch+1}/{num_epochs} completed.")
    print(f"Validation Precision: {val_precision:.4f}")
    print(f"Validation Recall: {val_recall:.4f}")
    print(f"Validation F1 Score: {val_f1:.4f}")
